# Import dependencies

In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, extract
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to database

In [3]:
# Connect to sqlite database using SQLAlchemy ORM
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
Measurement = Base.classes.measurement
Station = Base.classes.station

In [4]:
session = Session(engine)

## Temperature Analysis II

In [5]:
def calc_temps(start_date, end_date):
    pass
    """Calculate minimum (TMIN), average (TAVE), and maximum (TMAX) 
        temperatures for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    # TODO: write a function that meets the requirements specified above

In [10]:
def calc_temps(start_date, end_date):
    results = session.query(Measurement.date, func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        group_by(Measurement.date).\
            filter(Measurement.date >= start_date).\
                filter(Measurement.date <= end_date)
    
    df = pd.read_sql(results.statement, results.session.bind)
    
    return df

In [11]:
calc_temps("2016-08-01", "2016-08-07")

,date,min_1,avg_1,max_1
0,2016-08-01,73.0,76.166667,78.0
1,2016-08-02,72.0,76.571429,80.0
2,2016-08-03,74.0,78.428571,82.0
3,2016-08-04,74.0,78.000000,81.0
4,2016-08-05,75.0,78.500000,81.0
5,2016-08-06,76.0,78.166667,80.0
6,2016-08-07,77.0,78.833333,83.0


In [ ]:
# Using your function, calculate the min, avg, and max for the matching dates
# from the previous year

In [ ]:
# Plot results as a bar chart per instructions in README

## Daily Rainfall Average

In [ ]:
# Query rainfall per weather station for previous year, sorted by precipitation
# Include station, name, latitude, longitude, and elevation


## Daily Temperature Normals

In [ ]:
def daily_normals(date):
    """Calculate Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals: tmin, tavg, and tmax
    
    """
    
    # TODO: write a function that meets the requirements specified above


In [ ]:
# Calculate the daily normals for each day of your trip (inclusive)

start_date = '2017-08-01'
end_date = '2017-08-07'


In [ ]:
# Load results into a dataframe per README instructions

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`

## Close Session